In [1]:
import os
os.environ['YT_DEST'] = '/home/kwoksun2/anaconda2/pkgs/yt-3.3.5-np111py27_2/'
import numpy as np
from dengo.chemical_network import \
    ChemicalNetwork, \
    reaction_registry, \
    cooling_registry, species_registry
import dengo.primordial_rates, dengo.primordial_cooling
from dengo.chemistry_constants import tiny, kboltz, mh
import yt
import yt.units as u
import numpy
from IPython.display import Image
import matplotlib.pyplot as plt
import pickle
import pyximport
import matplotlib.colors as colors
import h5py
import timeit

pyximport.install(setup_args={"include_dirs":np.get_include()},
                      reload_support=True, inplace=True)
loc_pwd = "/home/kwoksun2/grackle/src/python/examples/"
def save_obj(obj, name ):
    with open(loc_pwd+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(loc_pwd + name + '.pkl', 'rb') as f:
        return pickle.load(f)

 using cli
 using CLI for selections


In [2]:
def Init_values(temperature, density, n_species = 6):
    """ Create a initial value dictionary,
        for a given temperature, density, number of species
    Args:
        temperature -- in Kelvin
        density     -- in amu / cm**3
    
    Returns:
        init_values: initial value dictionary with 
                     self-consistent energy/ electron density
        primordial : chemical_network classes
        
    """
    
    # initialize and setup the network
    dengo.primordial_rates.setup_primordial()
    primordial = ChemicalNetwork()
        
    if n_species == 9:
        for i in range(23):
            try:
                primordial.add_reaction("k{0:02d}".format(i+1))
            except:
                pass
    else:
        for i in range(6):
            try:
                primordial.add_reaction("k{0:02d}".format(i+1))
            except:
                pass        
    
    # the temperature array required to interpolates the rates
    primordial.init_temperature((1e0, 1e5))

    tiny = 1.0e-20

    # init_array are is in fractional abundances
    init_array = density
    init_values = dict()
    
    if n_species == 6:
        # 6-species model
        init_values["He_1"]    = density * (1.0 - 0.76) /2.
        init_values["He_2"]    = density * (1.0 - 0.76) /2.      
        init_values["He_3"]    = density * np.array([tiny])       
        init_values["H_1"]     = density *  (0.76)  /2.
        init_values['H_2']     = density *  (0.76)  /2.
    else:    
        # 9-species model
        init_values["He_1"]    = density * (1.0 - 0.76) /2.0
        init_values["He_2"]    = density * (1.0 - 0.76) /2.0  
        init_values["He_3"]    = np.array([tiny])           
        init_values["H_1"]     = density *  (0.76)  /2.
        init_values['H_2']     = np.array([tiny])

        init_values["H_m0"]    = np.array([tiny]) 
        init_values["H2_1"]    = density *  (0.76)  /2. 
        init_values["H2_2"]    = np.array([tiny])
    
    # now everything in mass density
    init_values['de'] = primordial.calculate_free_electrons(init_values)
    # one signle value: again mass density
    init_values['density'] = primordial.calculate_total_density(init_values)
    number_density = primordial.calculate_number_density(init_values)
        
    num_den = {}
    total_num_den = 0.0
    total_den = 0.0
    for sp in primordial.required_species:
        try:
            num_den[sp.name] = init_values[sp.name]/ sp.weight
        except:
            pass

    # set up initial temperatures values used to define ge
    init_values['T'] = temperature
    
    # calculate gammaH2
    x = 6100.0/temperature
    gammaH2 = 2.0 / (5.0 + 2.0*x*x / (numpy.exp(x) - 1 )**2.0 ) + 1
    
    gamma_factor = primordial.gamma_factor().subs(num_den).subs({'gammaH2': gammaH2 , 'gamma': 5./3.,'T': temperature })
    
    ge  = ((temperature *  kboltz) *gamma_factor
                         / (init_values['density'] * mh  ))
    
    T = init_values['density']*ge*mh / kboltz / gamma_factor
    init_values['ge'] = numpy.array( [numpy.float64(ge)] )
    
    return init_values, primordial

In [3]:
init, primordial = Init_values(np.array([1000.0]), np.array([1e5]), n_species = 6)

Adding reaction: k01 : 1*H_1 + 1*de => 1*H_2 + 2*de
Adding reaction: k02 : 1*H_2 + 1*de => 1*H_1
Adding reaction: k03 : 1*He_1 + 1*de => 1*He_2 + 2*de
Adding reaction: k04 : 1*He_2 + 1*de => 1*He_1
Adding reaction: k05 : 1*He_2 + 1*de => 1*He_3 + 2*de
Adding reaction: k06 : 1*He_3 + 1*de => 1*He_2


In [ ]:
for cooling_action in cooling_registry:
    k = cooling_registry[cooling_action]
    if (k.species).issubset( primordial.required_species ):
        if k.name != "cie_cooling":
            print(k.name)
            print(k.equation)
            primordial.add_cooling(cooling_action)
            print('---------------------------')

In [ ]:
def network_write_solver( network, init, solver_name = "BE_chem_solve", name = "sunny" ):
    """Generate the solver templates and import the modules
       for a given solver, and chemical network
    Args:
        network:     chemical_network class, 
                     contains infos of cooling actions, 
                     reactions to be included in the solver
        init:        initial conditions
        solver_name: the range of solver callable
        
        Names of Exisiting Solver Templates:
        -- sundials_CVSpils
        -- sundials_onestep_run
        -- BE_chem_solve (default)
        
    Returns:
        solver_run: the module generated from the templates, 
                    and it's ready to run
    """
    
    if solver_name == "sundials_CVSPils":
        network.write_solver("sundials_CVSpils", output_dir = ".",
                                   solver_template = "sundials_CVSpils",
                                   ode_solver_source = "cvodes_solver_CVSpils.C",
                                    init_values=init,
                                    input_is_number=False)


        sundials_cvspils_run = pyximport.load_module("sundials_CVSpils_run",
                                    "sundials_CVSpils_solver_run.pyx",
                                    build_inplace = True, pyxbuild_dir = "_dengo_temp")
        return sundials_cvspils_run
    elif solver_name == "sundials_onestep_run":
        network.write_solver("sundials_onestep", output_dir = ".",
                           solver_template = "sundials_onestep",
                           ode_solver_source = "cvodes_solver_onestep.C",
                            init_values=init,
                            input_is_number=False)
        sundials_onestep_run = pyximport.load_module("sundials_onestep_run",
                            "sundials_onestep_solver_run.pyx",
                            build_inplace = True, pyxbuild_dir = "_dengo_temp")
        return sundials_onestep_run
    else:
        network.write_solver(name, output_dir = ".",
                                    init_values=init,
                                    input_is_number=False)

        sunny_solver_run = pyximport.load_module("{}_solver_run".format(name),
                                        "{}_solver_run.pyx".format(name),
                                        build_inplace = True, pyxbuild_dir = "_dengo_temp")
        return sunny_solver_run

In [ ]:
my_solver_name = "hahaha_cvspils"

os.system('mkdir {}'.format(my_solver_name))


primordial.write_solver(my_solver_name, output_dir = "./{0}".format(my_solver_name),
                           solver_template = "cvspils/sundials_CVSpils",
                           ode_solver_source = "cvodes_solver_CVSpils.C",
                            init_values=init,
                            input_is_number=False)

In [ ]:
os.chdir("../")

In [ ]:
os.getcwd()

pyximport.install(setup_args={"include_dirs":np.get_include()},
                      reload_support=True, inplace=True)

In [ ]:
file_path = os.path.join(my_solver_name, "{}_solver_run.pyx".format(my_solver_name))
print(file_path)
sundials_cvspils_run = pyximport.load_module("{0}_run".format(my_solver_name),
                            "{0}_solver_run.pyx".format(my_solver_name),
                            pyxbuild_dir = "_dengo_temp")

In [ ]:
%%time
sundials_cvspils_run.run_hahaha_cvspils(init, 1e10, niter=10)

In [ ]:
primordial.write_solver("sundials_onestep", output_dir = ".",
                           solver_template = "cvdls/sundials_onestep",
                           ode_solver_source = "cvodes_solver_onestep.C",
                            init_values=init,
                            input_is_number=False)
sundials_onestep_run = pyximport.load_module("sundials_onestep_run",
                            "sundials_onestep_solver_run.pyx",
                             pyxbuild_dir = "_dengo_temp")

In [ ]:
%%time
sundials_onestep_run.run_sundials_onestep(init, 1e10, niter=10)

In [ ]:
%%time
primordial.write_solver("sundials_onestep", output_dir = ".",
                           solver_template = "sundials_onestep",
                           ode_solver_source = "cvodes_solver_onestep.C",
                            init_values=init,
                            input_is_number=False)
sundials_onestep_run = pyximport.load_module("sundials_onestep_run",
                            "sundials_onestep_solver_run.pyx",
                            build_inplace = True, pyxbuild_dir = "_dengo_temp")
rv, rv_int_onestep = sundials_onestep_run.run_sundials_onestep(init, 3e5, niter=1e4)

In [ ]:
import pstats, cProfile

import pyximport
pyximport.install()

import calc_pi

cProfile.runctx("sundials_onestep_run.run_sundials_onestep(init,1e10, niter=1e4)", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

In [ ]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

def make_plots( network, init, rv_int, solver_name = 'sundials', t_min = 1e0, t_max = 1e11, plot_fH = False ):
    
    rv_arr = rv_int
    
    mask = rv_arr['successful']
    t    = rv_arr['t'][mask]
    dt   = rv_arr['dt'][mask]

    plt.clf()
    skip_list = ['dt' ,'successful', 't','Ju', 'Jac']
    hydrogen_list = ['H_1','H_2','H2_1','H2_2','H_m0']
    
    
    # Calculate total electron density 
    # with the abundances from other species
    total_e = numpy.zeros((len(t)))
    for j in network.required_species:
        if j.name in rv_arr.keys() and j.name not in skip_list and j.name != 'dt':
            
            try:
                total_e += rv_arr[j.name][0][mask]  * j.free_electrons 
            except:
                pass
    
    de = rv_arr['de'][0][mask]
    # calculate total mass density of hydrogen
    # conservation of mass
    total_Hatoms = numpy.zeros((len(t)))
    for j in network.required_species:
        if j.name in hydrogen_list:
            total_Hatoms += rv_arr[j.name][0][mask] *j.weight
            

    f, axarr = plt.subplots(3, sharex=True, figsize=(10,15))
    
    ii = 0
    for i in sorted(rv_arr.keys()):
        if i not in skip_list:
            
            hi = rv_arr[i][0][mask]
            if t_min == 1e0:
                axarr[0].semilogx(t,numpy.array(hi)/init[i][0]+ii, label=i) 
                ii+=1
            else:
                idx = find_nearest(t , t_min)
                axarr[0].semilogx(t[idx:] ,numpy.array(hi[idx:])/hi[idx] , label=i) 
                ii+=1



    #h1 = rv_arr['H_1'][0][mask]

    
    T0 = rv_arr['T'][0][mask][0]
    axarr[0].set_title('{}:  Temperature: {} K     Density: {} per cc '.format(solver_name, T0 , total_Hatoms[0]   ))
    axarr[0].set_xlabel('Time (s)')
    axarr[0].set_ylabel('Species abundances (normalized by initial values)')
    
    dt = rv_arr['dt'][mask]
 #   if t_min == 1e0:
 #       axarr[0].loglog(t, dt / dt[0], label=i) 
 #   else:
 #       idx = find_nearest(t , t_min)
 #       axarr[0].loglog(t[idx:] ,dt[idx:] / dt[idx], label=i) 
                
    

    de = rv_arr['de'][0][mask]
    axarr[0].loglog(t, total_e/de, label='de normalized by those from species', ls = '--', color = 'r')

    axarr[0].legend(loc='best',fontsize='x-small')
    # plt.savefig(con_not_con+'-species-norm-nHe20.png')


    for i in sorted(rv_arr.keys()):
        if i not in skip_list:
            hi = rv_arr[i][0][mask]
            axarr[1].loglog(t,hi, label=i) 


    axarr[1].set_xlabel('Time (s)')
    axarr[1].set_ylabel('Species abundances')

    axarr[1].legend(loc='best',fontsize='x-small')


    

    de = rv_arr['de'][0][mask]
    axarr[2].semilogx(t, (de/total_e) , label='de normalized by those from species', ls = '--', color = 'r')

    axarr[2].semilogx(t, ( total_Hatoms )/ total_Hatoms[0], label = 'mass density of H atoms' )
    
    
    print('de conserved?   ', de[-1]/total_e[-1])
    print('mass conserved? ' , ( total_Hatoms[-1] )/ total_Hatoms[0])
    
    
    if plot_fH:
        axarr[2].plot(t, h1/total_Hatoms, label="f(H)" )
        print('final f(H) = {}'.format(h1[-1]/total_Hatoms[-1]) )
    axarr[2].set_xlabel('Time (s)')
    axarr[2].set_ylabel('mass ratio of Hydrogen atom (fH)')
    axarr[2].legend()
    
    axarr[2].set_xlim((t_min, t_max))
    
    T0 = rv_arr['T'][0][0]
    f.savefig(solver_name+'_nH={:.2E}_T={:.1E}.png'.format(total_Hatoms[0], T0) )
    plt.close(f)
    return solver_name+'_nH={:.2E}_T={:.1E}.png'.format(total_Hatoms[0], T0)

In [ ]:
mask = rv_int_cvspils['successful']
t_arr = rv_int_cvspils['t'][mask]
png_name = make_plots( primordial, init, rv_int_cvspils, solver_name = 'cvspils', t_min = t_arr[0], t_max = t_arr[-1])
Image(png_name)

In [ ]:
mask = rv_int_onestep['successful']
t_arr = rv_int_onestep['t'][mask]
png_name = make_plots( primordial, init, rv_int_onestep, solver_name = 'onestep', t_min = t_arr[0], t_max = t_arr[-1])
Image(png_name)

In [ ]:
rv_int_cvspils.keys()

In [ ]:
def interp_species( rv_int, key , t_arr):
    rv_arr = rv_int
    
    mask = rv_arr['successful']
    t    = rv_arr['t'][mask]
    dt   = rv_arr['dt'][mask]

    tmp = rv_arr[key][0][mask]
    print(key, min(tmp))
    ans = numpy.interp(t_arr, t, tmp)
    
    return ans

In [ ]:
mask1 = rv_int_cvspils['successful']
mask2 = rv_int_onestep['successful']
maxt1 = max(rv_int_cvspils['t'][mask1]) 
maxt2 = max(rv_int_onestep['t'][mask2])

maxt = min([maxt1,maxt2])

t_arr = numpy.logspace(0,numpy.log10(maxt),100)

In [ ]:
for key_plot in rv_int_cvspils.keys():
    try:
#        if key_plot not in ["H_2", "H_m0", "H2_2", "de", "He_3","He_2"]:
        y_spils = interp_species( rv_int_cvspils, key_plot, t_arr)
        y_dls = interp_species( rv_int_onestep, key_plot, t_arr)
        plt.semilogx(t_arr,y_spils/y_dls,label=key_plot)
    
    except:
        pass
#plt.ylim(0.9,1.1)
plt.legend(loc='upper left')
plt.show()


In [ ]:
maxt2/1e9

In [ ]:
mask = rv_int_cvspils['successful']

In [ ]:
import yt.units as u

In [ ]:
T = 1.0 / numpy.sqrt( u.G*1e20*(u.cm**-3)*u.mass_hydrogen)

In [ ]:
T/1e5

In [ ]:
def time_taken_cvspils(Tdim = 50 ,Ddim = 50 ,total_t = 1e10, n_species=6):
    time_taken = []
    success = []
    
    HI_arr = []
    HII_arr = []
    HeI_arr = []
    HeII_arr = []
    HeIII_arr = []
    de_arr = []
    ge_arr = []
    T_arr = []
    total_t_arr = []

    temp_list = numpy.logspace(2,4,Tdim)
    den_list  = numpy.logspace(0,18,Ddim)
    
    temp_2d, den_2d = numpy.meshgrid(temp_list,den_list)
    den_temp_pair2d = numpy.dstack( (temp_2d,den_2d)  )

    den_temp_pair2d = ( numpy.reshape(den_temp_pair2d, (Tdim*Ddim,2) )  )

    temp_arr = []
    den_arr = []
    
    # initialize data dictionary
    init, primordial = Init_values(np.array([1000]), np.array([1e10]), n_species = n_species)
    rv, rv_int_sundials = sundials_cvspils_run.run_sundials_CVSpils(init, total_t, niter=1e5)
    data_dict = {}
    for sp in rv_int_sundials.keys():
        data_dict[sp] = []
        
    
    for temp, den in den_temp_pair2d:
        
        # initial conditions for a given temperature and density
        init, primordial = Init_values(np.array([temp]), np.array([den]), n_species = n_species)
        total_t = calc_fftime(den).v
        # Calculate the time it takes
        tic=timeit.default_timer()
        rv, rv_int_sundials = sundials_cvspils_run.run_sundials_CVSpils(init, total_t, niter=2e5)
        toc=timeit.default_timer()
        time_taken.append(toc - tic)

        temp_arr.append(temp)
        den_arr.append(den)

        flag = rv_int_sundials['successful']
        try:
            final_t = rv_int_sundials['t'][flag][-1]
        except:
            final_t = 0.0
        success.append(final_t)
        total_t_arr.append(total_t)
        for sp in rv_int_sundials.keys():
            if sp not in ["successful", 'dt', 't']:
                data_dict[sp].append( rv_int_sundials[sp][0][flag][-1] )
            else:
                data_dict[sp].append( rv_int_sundials[sp][flag][-1] )
    success = numpy.array(success)
    time_taken = numpy.array(time_taken)
    total_t_arr = numpy.array(total_t_arr)
    temp_arr = numpy.array(temp_arr)
    den_arr = numpy.array(den_arr)
    
    filename = 'cvspils_final_%d.hdf5' %(n_species)
    try:
        f = h5py.File(filename, 'w')
    except:
        f = h5py.File(filename)
        f.close()
        f = h5py.File(filename, 'w')
    for key in data_dict.keys():
        dset = f.create_dataset(key, data = data_dict[key])
    dset = f.create_dataset( "density", data = den_arr )
    dset = f.create_dataset( "temperature", data = temp_arr )
    dset = f.create_dataset( "success", data = success )
    dset = f.create_dataset( "time_taken", data = time_taken)
    dset = f.create_dataset( "total_t", data = total_t_arr)
    
    dset = f.create_dataset( "Tdim", data = (Tdim))
    dset = f.create_dataset( "Ddim", data = (Ddim))
    
    f.close()
    
    return success, time_taken, temp_arr, den_arr, filename

In [ ]:
# filename = 'sundials_final_%d.hdf5' %(9)
# import os
# os.system('rm {}'.format(filename))



niter = 1
Tdim = 20
Ddim = 20
total_t = 1e10

success, time_taken, temp_arr, den_arr, filename_sun = time_taken_cvspils(Tdim = Tdim ,
                                                                           Ddim = Ddim ,
                                                                           total_t = total_t,
                                                                           n_species=9)

In [ ]:
def calc_fftime(den):
    mu = 1.0
    rho = mu* u.mass_hydrogen *den *(u.cm**-3)
    tff = numpy.sqrt(1.0 / u.G / rho).in_units('s')
    return tff

In [ ]:
def plot_performance(Tdim, Ddim, temp_arr,den_arr, success, time_taken, title=None, performance=True, log=True, total_t = total_t):

    success = numpy.array(success)
    time_run = numpy.copy(success)
    i1 = success >= total_t*(0.9)
    i2 = ~i1
    
    success[i1] = 1
    success[i2] = 0

    success_mat = success.reshape((Ddim,Tdim))
    total_t_mat = total_t.reshape((Ddim,Tdim))

#     temp_2d = temp_arr.reshape(Ddim + 1,Tdim + 1)
#     den_2d = den_arr.reshape(Ddim + 1,Tdim + 1)
    
    temp_2d, den_2d = numpy.meshgrid(temp_arr, den_arr)
    
    time_taken_mat = numpy.array(time_taken).reshape((Ddim,Tdim))
    time_run_mat = numpy.array(time_run).reshape((Ddim,Tdim))
    
    if performance:
        fig, ax = plt.subplots(1,2, figsize = (16,8))
        
        pcm = ax[0].pcolor( temp_2d, den_2d, time_run_mat/total_t_mat)
        fig.colorbar(pcm, ax=ax[0], extend='max')


        ax[0].set_xscale('log')
        ax[0].set_yscale('log')
        ax[0].set_xlabel('Temperature (K)')
        ax[0].set_ylabel('Density ($cm^{-3}$)')
        
        if log:
            pcm = ax[1].pcolor( temp_2d, den_2d, time_taken_mat,
                              norm=colors.LogNorm(vmin=time_taken_mat.min(), vmax=time_taken_mat.max()))
        else:
            pcm = ax[1].pcolor( temp_2d, den_2d, time_taken_mat)
        fig.colorbar(pcm, ax=ax[1], extend='max')

        ax[1].set_xscale('log')
        ax[1].set_yscale('log')
        ax[1].set_xlabel('Temperature (K)')
        ax[1].set_ylabel('Density ($cm^{-3}$)')
        if title != None:
            ax[0].set_title(title+' time evolved ')
            ax[1].set_title(title+' run time')

        plt.show()
    else:
        fig, ax = plt.subplots(1,1, figsize = (8,8))
        if log:
            pcm = ax.pcolor( temp_2d, den_2d, time_taken_mat,
                              norm=colors.LogNorm(vmin=time_taken_mat.min(), vmax=time_taken_mat.max()))
        else:        
            pcm = ax.pcolor( temp_2d, den_2d, time_taken_mat)
        fig.colorbar(pcm, ax=ax, extend='max')

        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel('Temperature (K)')
        ax.set_ylabel('Density ($cm^{-3}$)')
        if title != None:
            ax.set_title(title+' abundances')

        plt.show()

In [ ]:
filename_sun = 'cvspils_final_%d.hdf5' %(9)
f = h5py.File(filename_sun,'r')

Tdim = f['Tdim'].value
Ddim = f['Ddim'].value
print(Tdim, Ddim)
temp_arr = f['temperature'].value
den_arr = f['density'].value
success = f['success'].value
time_taken = f['time_taken'].value
total_t = f['total_t'].value
f.close()

filename = 'sundials_final_%d.hdf5' %(9)
fold = h5py.File(filename,'r')

time_taken_fold = fold['time_taken'].value
total_t_fold = fold['total_t'].value
success_fold = fold['success'].value
fold.close()


logmin_temp, logmax_temp = numpy.log10(temp_arr.min()), numpy.log10(temp_arr.max())
logmin_den , logmax_den  = numpy.log10(den_arr.min()) , numpy.log10(den_arr.max())

dlogtemp = ( logmax_temp - logmin_temp )/ Tdim /2.0
dlogden  = ( logmax_den  - logmin_den )/ Ddim / 2.0
temp_arr = numpy.logspace( logmin_temp - dlogtemp , logmax_temp + dlogtemp , Tdim + 1 )
den_arr  = numpy.logspace( logmin_den - dlogden, logmax_den + dlogden, Ddim + 1 )

plot_performance(Tdim, Ddim, temp_arr,den_arr, success, time_taken_fold/time_taken, title="DLS / cvspils", log=True,total_t = total_t)

In [ ]:
filename_sun = 'cvspils_final_%d.hdf5' %(9)
f = h5py.File(filename_sun,'r')

Tdim = f['Tdim'].value
Ddim = f['Ddim'].value
print(Tdim, Ddim)
temp_arr = f['temperature'].value
den_arr = f['density'].value
success = f['success'].value
time_taken = f['time_taken'].value
total_t = f['total_t'].value


filename = 'sundials_final_%d.hdf5' %(9)
f_sun = h5py.File(filename,'r')

Tdim = f_sun['Tdim'].value
Ddim = f_sun['Ddim'].value
den_arr = f_sun['density'].value
success = f_sun['success'].value

time_taken = f_sun['time_taken'].value
total_t = f_sun['total_t'].value
success = f_sun['success'].value



logmin_temp, logmax_temp = numpy.log10(temp_arr.min()), numpy.log10(temp_arr.max())
logmin_den , logmax_den  = numpy.log10(den_arr.min()) , numpy.log10(den_arr.max())

dlogtemp = ( logmax_temp - logmin_temp )/ Tdim /2.0
dlogden  = ( logmax_den  - logmin_den )/ Ddim / 2.0
temp_arr = numpy.logspace( logmin_temp - dlogtemp , logmax_temp + dlogtemp , Tdim + 1 )
den_arr  = numpy.logspace( logmin_den - dlogden, logmax_den + dlogden, Ddim + 1 )

for species in f.keys():
    if species not in ["Ddim", "Tdim","dt","success",'t']:
        abundance = f_sun[species].value
        abundance_spils = f[species].value
        print(species)
        plot_performance(Tdim, Ddim, temp_arr,den_arr, success, abundance/abundance_spils, title=species, performance = False,total_t=total_t)    
f_sun.close()
f.close()

In [ ]:
filename_sun = 'cvspils_final_%d.hdf5' %(9)
f = h5py.File(filename_sun,'r')
total_t = f['total_t'].value

In [ ]:
total_t

In [ ]:
f_sun[species].value

In [ ]:
((u.G/(u.c**2) *(3e-6/u.s)*3./4./ numpy.pi*(u.Msun)*(u.rsun**2) / 15/(u.AU**3))*u.year*u.year).in_units('s')

In [5]:
!rm -r happy/

In [4]:
import generate_primordial_templates

reload(generate_primordial_templates)

<module 'generate_primordial_templates' from 'generate_primordial_templates.pyc'>

In [6]:
hiii = generate_primordial_templates.network_write_solver(primordial, init, ode_solver_name="CVSpils",name='cry')

In [ ]:
generate_primordial_templates.network_write_solver?

In [7]:
%%time
hiii.run_cry(init, 1e10, niter=1)

initial  temperature: 1.000E+03
End in 1 iterations: 0.00000e+00 / 1.00000e+10 (1.00000e+10)
CPU times: user 0 ns, sys: 15.6 ms, total: 15.6 ms
Wall time: 4.92 ms


({'H_1': array([ 38000.]),
  'H_2': array([ 38000.]),
  'He_1': array([ 12000.]),
  'He_2': array([ 12000.]),
  'He_3': array([  1.00000000e-15]),
  'de': array([ 40698.70655375]),
  'ge': array([  1.51340699e+11])},
 {'H_1': array([[ 0.]]),
  'H_2': array([[ 0.]]),
  'He_1': array([[ 0.]]),
  'He_2': array([[ 0.]]),
  'He_3': array([[  2.49837481e-16]]),
  'T': array([[ nan]]),
  'de': array([[ 0.]]),
  'dt': array([  1.00000000e+10]),
  'ge': array([[  1.51340699e+11]]),
  'successful': array([False], dtype=bool),
  't': array([ 0.])})

##### init 

In [9]:
eval("primordial.required_"+"species")

{Species: H_1,
 Species: H_2,
 Species: He_1,
 Species: He_2,
 Species: He_3,
 Species: de,
 Species: ge}